# Tread safe hashtable stress test

In [4]:
import time
import random
import threading

from project.task5.hash_table import HashTable
from project.task6.parallel_hash_table import HashTable as BaselineHashTable

TOTAL_OPS = 5_000_000
READ_RATIO = 0.80
NUM_THREADS = 8
KEY_SPACE = 20000

import sources directory


## Creating operations set

In [14]:
def generate_operations(total_ops, read_ratio, key_space, seed=0):
    rng = random.Random(seed)
    ops = []
    for _ in range(total_ops):
        if rng.random() < read_ratio:
            ops.append(('get', rng.randrange(0, key_space), None))
        else:
            # writes: either set or delete; make set more probable
            if rng.random() < 0.9:
                ops.append(('set', rng.randrange(0, key_space), rng.randrange(1, 1 << 30)))
            else:
                ops.append(('del', rng.randrange(0, key_space), None))
    return ops

## Operations execution rules

In [11]:
def run_ops_on_table(table, ops, num_threads):
    # split ops among threads
    chunk = len(ops) // num_threads
    threads = []
    start_t = time.time()
    def worker(subops):
        for op, k, v in subops:
            try:
                if op == 'get':
                    try:
                        _ = table[k]
                    except KeyError:
                        pass
                elif op == 'set':
                    table[k] = v
                elif op == 'del':
                    try:
                        del table[k]
                    except KeyError:
                        pass
            except IndexError:
                pass
            except Exception as e:
                print("Exception during op:", e)


    for i in range(num_threads):
        s = i * chunk
        e = len(ops) if i == num_threads - 1 else s + chunk
        t = threading.Thread(target=worker, args=(ops[s:e],))
        threads.append(t)
        t.start()

    for t in threads:
        t.join()
    end_t = time.time()
    return end_t - start_t

## Treadsafe hashtable run

In [15]:
ops = generate_operations(TOTAL_OPS, READ_RATIO, KEY_SPACE, seed=12345)
print("Operations generated:", len(ops))
print("Running stress test on thread-safe hash table...")
ts_table = HashTable(initial_capacity=64)
t_ts = run_ops_on_table(ts_table, ops, NUM_THREADS)
print(f"Thread-safe table: time {t_ts:.2f}s, final size {len(ts_table)}")

Operations generated: 5000000
Running stress test on thread-safe hash table...
Thread-safe table: time 2.18s, final size 17926


## Baseline hashtable run

In [ ]:
# Работал дольше часа и не завершился
ops2 = generate_operations(TOTAL_OPS, READ_RATIO, KEY_SPACE, seed=12345)
base_table = BaselineHashTable(initial_capacity=64)
t_base = run_ops_on_table(base_table, ops2, NUM_THREADS)
print(f"Baseline table: time {t_base:.2f}s, final size {len(base_table)}")